# Import necessary packages

In [ ]:
import qiskit as q
import pec_module as pec
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

from numpy.linalg import inv
from itertools import product
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError

# Connect to ibm select the device to mimic the error

In [2]:
token = "PJubuPXreZKAxan0yV6OLEIkdKipFYGgvvkGQSZAtpMU"
instance = 'crn:v1:bluemix:public:quantum-computing:us-east:a/f071734952cb4c9993a642d0a87d18bb:392306ae-fe55-475c-a8c5-978d0ba1871e::'
QiskitRuntimeService.save_account(
        channel="ibm_quantum_platform",
        token=token,
        instance=instance,
        set_as_default=True,
        overwrite=True
    )
device_name = "kingston"
dev = "ibm_" + device_name
service = QiskitRuntimeService()
device = service.backend(dev)
noise_model = NoiseModel.from_backend(device)

In [ ]:
# # # Load saved credentials
# # service = QiskitRuntimeService()

# # #For real Device
# # device = service.backend("ibm_kyiv")
# # noise_model = NoiseModel.from_backend(device)

# # Get basis gates from noise model
# basis_gates = noise_model.basis_gates

# # Get coupling map from backend
# coupling_map = [[0, 1], [1, 2], [3, 2], [3, 4]]

# #For Aer
# # backend = AerSimulator(noise_model=noise_model,
# #                        coupling_map=coupling_map,
# #                        basis_gates=basis_gates)
# backend = AerSimulator()

# backendqubitNum = backend.num_qubits
# print(backendqubitNum)

In [6]:
# 建立空的 noise model
# noise_model = NoiseModel()

# # 加入 CNOT gate 的 depolarizing noise（以 (0,1) 為例）
# cx_error = depolarizing_error(param=0.05, num_qubits=2)  # 1% depolarizing error
# noise_model.add_quantum_error(cx_error, 'cx', [0, 1])

# 加入 measurement 的 readout error（以 qubit 0 為例）
# readout_probs = [[0.95, 0.05], [0.02, 0.98]]  # 機率轉換矩陣
# readout_error = ReadoutError(readout_probs)
# noise_model.add_readout_error(readout_error, [0])
# noise_model.add_readout_error(readout_error, [1])

# 設定模擬器
backend = AerSimulator(noise_model=noise_model)
backendqubitNum = backend.num_qubits
print(backendqubitNum)

34


In [ ]:
# # For NoisyModel Check

# from pprint import pprint
# print("\n[Local Quantum Errors]")
# pprint(noise_model._local_quantum_errors)
# print("=== All gate errors in the noise model ===")
# for gates, errors in noise_model._local_quantum_errors.items():
#     # print(gates, '\n', errors)
#     if gates == 'cz':
#         print('Gate: ', gates)
#         for qubits, err in errors.items():
#             print(qubits, ': ', err)
        # print(errors)
#     for err in errors:
#         print(err)
# print(noise_model)

# print("\n=== All readout errors in the noise model ===")
# for qubits, err in noise_model._local_readout_errors.items():
#     print(f"Qubits: {qubits}, Error: {err}")

# Do 2-qubits measurement PEC

In [3]:
#Build Four Initial State |0>, |1>, |0>+|1>, |0>-i|1>
qreg = QuantumRegister(2, 'q')
creg = ClassicalRegister(2, 'c')
q0_init = {}
q1_init = {}

for i in range (2):
    #rho0 state |0>
    InitCirZero = QuantumCircuit(qreg, creg)
    #rho1 state |1>
    InitCirOne = QuantumCircuit(qreg, creg)
    InitCirOne.x(i)
    #rho2 state |0>+|1>
    InitCirPlus = QuantumCircuit(qreg, creg)
    InitCirPlus.h(i)
    #rho3 state |0>+i|1>
    InitCirRight = QuantumCircuit(qreg, creg)
    InitCirRight.h(i)
    InitCirRight.s(i)
    
    if i == 0:
        q0_init = {'q0zero':InitCirZero, 'q0one':InitCirOne, 'q0plus':InitCirPlus, 'q0right':InitCirRight}
    else:
        q1_init = {'q1zero':InitCirZero, 'q1one':InitCirOne, 'q1plus':InitCirPlus, 'q1right':InitCirRight}

# Store the initial circuits in a dictionary
InitialState = {}
for q0, q0state in q0_init.items():
    for q1, q1state in q1_init.items():
        initcomb = q.circuit.QuantumCircuit.compose(q0state, q1state)
        InitialState.update({f'{q0}_{q1}':initcomb})

In [4]:
# Store the measurement circuits in a dictionary
Measurement = {}
pau = ['I', 'X', 'Y', 'Z']
for i in pau:
    for j in pau:
        # meascomb = Pauli('I' * (backendqubitNum-2) + j + i) 
        meascomb = Pauli(j + i) 
        Measurement.update({f'meas{j}{i}':meascomb})
print(Measurement)

{'measII': Pauli('II'), 'measXI': Pauli('XI'), 'measYI': Pauli('YI'), 'measZI': Pauli('ZI'), 'measIX': Pauli('IX'), 'measXX': Pauli('XX'), 'measYX': Pauli('YX'), 'measZX': Pauli('ZX'), 'measIY': Pauli('IY'), 'measXY': Pauli('XY'), 'measYY': Pauli('YY'), 'measZY': Pauli('ZY'), 'measIZ': Pauli('IZ'), 'measXZ': Pauli('XZ'), 'measYZ': Pauli('YZ'), 'measZZ': Pauli('ZZ')}


In [7]:
job = {}
estimator = Estimator(mode=backend)
estimator.options.default_shots = 10000
for MeasName, Measure in Measurement.items():
    for InitName, Initial in InitialState.items():
        CirTran = q.compiler.transpile(Initial, backend=backend, optimization_level=0)
        job.update({f'{InitName}_{MeasName}':estimator.run([(CirTran, Measure)])})

KeyboardInterrupt: 

In [ ]:
result_origin = []
for index, job_name in job.items():
    res = job_name.result()
    result_origin.append(res[0].data.evs)
result_origin

In [ ]:
g = np.array(result_origin).reshape(16, 16)
g

In [ ]:
#Create State Preparation Matrix A
A_single = np.array([[1, 1, 1, 1],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1],
                     [1,-1, 0, 0]])
A = np.kron(A_single, A_single)
#print(A)

#Calculate Readout Matrix by the quation B = g * A^-1
A_inv = inv(A)
#print(A_inv)
B = np.matmul(g, A_inv)
B_inv = inv(B)
# print(B_inv)

#Calculate observable X, Y, Z
a_i = np.array([[1, 0, 0, 0]])
a_x = np.array([[0, 1, 0, 0]])
a_y = np.array([[0, 0, 1, 0]])
a_z = np.array([[0, 0, 0, 1]])
a = {'I':a_i, 'X':a_x, 'Y':a_y, 'Z':a_z}
aa = {}
qq = {}
for f, first in a.items():
    for s, second in a.items():
        a_mix = np.kron(first, second)
        aa.update({f'a_{s}{f}':a_mix})
        q_mix = np.matmul(a_mix, B_inv)
        qq.update({f'{s}{f}':q_mix})
        
        # print(f'a_{f}{s}:', a_mix)
        # print(f'{f}{s}:', q_mix)
        # print('\n')
print(qq)

In [ ]:
#Observable = Oq1 Oq0
Observable = ['II', 'XI', 'YI', 'ZI', 'IX', 'XX', 'YX', 'ZX', 'IY', 'XY', 'YY', 'ZY', 'IZ', 'XZ', 'YZ', 'ZZ']
IdealMeasurement = {}
for obs in Observable:
    # IdealObservable = SparsePauliOp(['I' * (backendqubitNum)], coeffs=[qq[obs][0][0]])
    IdealObservable = SparsePauliOp(['II'], coeffs=[qq[obs][0][0]])
    for i in range(1, len(Observable)):
        # IdealObservable += SparsePauliOp([('I' * (backendqubitNum-2) + Observable[i])], coeffs=[qq[obs][0][i]])
        IdealObservable += SparsePauliOp([(Observable[i])], coeffs=[qq[obs][0][i]])
    IdealMeasurement.update({f'meas{obs}':IdealObservable})
IdealMeasurement

In [ ]:
len(IdealMeasurement)

# Do 2-qubits Cnot PEC

In [ ]:
#Build each set of circuits with Initialstates, cnot gate, measurements
#Cnot gate
Cnotgate = QuantumCircuit(qreg, creg)
Cnotgate.cx(0, 1)
Circuit = {}
for InitName, Initial in InitialState.items():
    circuit = q.circuit.QuantumCircuit.compose(Initial, Cnotgate)
    Circuit.update({f'{InitName}':circuit})
Circuit

In [ ]:
Circuit['q0plus_q1right'].draw(output='mpl') 

In [ ]:
estimator.options.default_shots = 1024
job_cnot = {}
for MeasName, Measure in IdealMeasurement.items():
    for CirName, Cir in Circuit.items():
        CirTran = q.compiler.transpile(Cir, backend=backend, optimization_level=0)
        job_cnot.update({f'{CirName}_{MeasName}':estimator.run([(CirTran, Measure)])})

In [ ]:
result_cnot_origin = []
for index, job_name in job_cnot.items():
    res = job_name.result()
    result_cnot_origin.append(res[0].data.evs)
g_cnot = np.array(result_cnot_origin).reshape(16, 16)
g_cnot

In [ ]:
#Calculate Readout Matrix by the quation Uerror = B^-1 * g_cnot * A^-1
ErrorCnot = np.matmul(np.matmul(B_inv, g_cnot), A_inv)
print(ErrorCnot)
ErrorCnot_inv = inv(ErrorCnot)
# print(ErrorCnot_inv)

In [ ]:
import numpy as np
import csv
# 將矩陣存成CSV檔案
filename = 'matrix_ErrorCnot.csv'
np.savetxt(filename, ErrorCnot, delimiter=",", fmt="%.10e")
print(f"矩陣已成功存成CSV檔案：{filename}")

# Build PTM of the Ideal Ucnot 

In [ ]:
#Construct Ideal Ucnot
pauli = ["I", "X", "Y", "Z"]
pauli_pair = {}
for i in pauli:
    for j in pauli:
        pauli_pair.update({f"{i}{j}":np.array(Pauli(f"{i}{j}").to_matrix())})

def evolution(gate, state):
    return np.matmul(np.matmul(gate, state), np.conj(gate).T)
        
Cnot = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0],
                 [0 ,0, 0, 1],
                 [0, 0, 1, 0]])
IdealCnot = np.ones((16,16),  dtype=complex)
i = 0
for index1, matrix1 in pauli_pair.items():
    j = 0
    for index2, matrix2 in pauli_pair.items():
        applycnot = evolution(Cnot, matrix1)
        element = np.matmul(matrix2, applycnot)
        IdealCnot[i][j] = np.round(np.trace(element)/4)
        j+=1
    i+=1 

print(IdealCnot)

In [ ]:
import numpy as np
import csv
# 將矩陣存成CSV檔案
filename = 'matrix_IdealCnot.csv'
np.savetxt(filename, IdealCnot, delimiter=",", fmt="%.10e")
print(f"矩陣已成功存成CSV檔案：{filename}")

In [ ]:
#Construct Error channel
LambdaError = np.matmul(inv(IdealCnot), ErrorCnot)
# print(np.round(LambdaError2))
print(LambdaError.real)

# Pauli Twirling

In [ ]:
pauli_transfer = {}
for channel, matrix in pauli_pair.items():
    new_matrix = evolution(Cnot, matrix)
    for new_channel, comp_matrix in pauli_pair.items():
        if np.array_equal(new_matrix, comp_matrix) or np.array_equal(new_matrix, -comp_matrix):
            pauli_transfer.update({channel:new_channel})
print(pauli_transfer)

In [ ]:
# Function to generate measurement circuits
def generate_pauli_circuits(index):
    circuits = {
        "I": QuantumCircuit(2, 2),
        "X": QuantumCircuit(2, 2),
        "Y": QuantumCircuit(2, 2),
        "Z": QuantumCircuit(2, 2),
    }
    # X-measurement
    circuits["X"].x(index)
    # Y-measurement
    circuits["Y"].y(index)
    # Z-measurement
    circuits["Z"].z(index)

    return circuits

# Store the measurement circuits in a dictionary
PauliTwirling = {}
for q0_label, q0 in generate_pauli_circuits(0).items():
    for q1_label, q1 in generate_pauli_circuits(1).items():
        pauli_comb = q.circuit.QuantumCircuit.compose(q0, q1)
        PauliTwirling.update({f"{q0_label}{q1_label}": pauli_comb})
print(PauliTwirling)

In [ ]:
Twirling_circuit_set = {}
for firstPauli, secondPauli in pauli_transfer.items():
    Twirling_circuit = {}
    for InitName, Initial in InitialState.items():
        AddPauli = q.circuit.QuantumCircuit.compose(Initial, PauliTwirling[firstPauli])
        AddCnot = q.circuit.QuantumCircuit.compose(AddPauli, Cnotgate)
        AddPauliInverse = q.circuit.QuantumCircuit.compose(AddCnot, PauliTwirling[secondPauli])
        Twirling_circuit.update({InitName:AddPauliInverse})
    Twirling_circuit_set.update({firstPauli:Twirling_circuit})

In [ ]:
Twirling_circuit_set['XY']['q0plus_q1right'].draw(output='mpl') 

In [ ]:
Twirling_job = {}
for pauli, circuit_set in Twirling_circuit_set.items():
    job = {}
    for MeasName, Measure in IdealMeasurement.items():
        for CirName, Cir in circuit_set.items():
            CirTran = q.compiler.transpile(Cir, backend=backend, optimization_level=0)
            job.update({f'{CirName}_{MeasName}':estimator.run([(CirTran, Measure)])})
    Twirling_job.update({pauli:job})

In [ ]:
Sum_twirling = np.zeros((16,16))
for pauli, job_set in Twirling_job.items():
    result_each_twirling = []
    for index, job_name in job_set.items():
        res = job_name.result()
        result_each_twirling.append(res[0].data.evs)
    g_twirling = (np.array(result_each_twirling).reshape(16, 16))
    # ErrorCnot_twirling = np.matmul(np.matmul(B_inv, g_twirling), A_inv)
    ErrorCnot_twirling = np.matmul(g_twirling, A_inv)
    Sum_twirling = Sum_twirling + ErrorCnot_twirling

Avg_twirling = Sum_twirling/16
Avg_gateError = np.matmul(inv(IdealCnot), Avg_twirling)
Avg_gateError
diag = np.diag(Avg_gateError)
diag

In [ ]:
# c = T * α -> α = T^-1 * c
c = np.diag(Avg_gateError)

P_mtrx = [Pauli(i+j).to_matrix() for i, j in product(pau, repeat=2)]
trnscoe = []
for bas in P_mtrx:
    bascoe = []
    for tw in P_mtrx:
        mult = np.matmul(np.matmul(tw, bas), tw)
        if np.array_equal(mult, bas):
            bascoe.append(1)
        elif np.array_equal(mult, -bas):
            bascoe.append(-1)
    trnscoe.append(bascoe)


alpha = np.matmul(inv(trnscoe), c)
inv_alpha = 1/alpha
inv_alpha2 = np.matmul(inv(trnscoe), 1/c)
print(alpha)
print(inv_alpha)
print(inv_alpha2)
sum(inv_alpha2)

In [ ]:
def OutputMatrix(matrix, filename):
    # 將每列轉為空格分隔的字串
    matrix_lines = [''.join(f"{val:>9.5f}" for val in row) for row in matrix]
    
    # 繪製圖片
    fig, ax = plt.subplots(figsize=(6, 3))
    ax.axis('off')  # 不顯示座標軸
    
    # 建立表格
    table_data = []
    for i in range(16):
        row = []
        for j in range(16):
            val = matrix[i][j]
            formatted = f"{val.real:.6f}"
            row.append(formatted)
        table_data.append(row)
    
    # 建立表格物件
    table = ax.table(cellText=table_data,  loc='center',  cellLoc='center')
    
    # 樣式調整
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            cell = table[i, j]
            val = matrix[i][j]
            
            # 主對角線粗體
            if 0.8 <= abs(val) < 1.2:
                cell.set_text_props(weight='bold')
    
            # 紅色標註：介於 0.01 ~ 0.1 或 -0.01 ~ -0.1
            if 0.01 <= abs(val) < 0.1:
                cell.set_text_props(color='red')
    
    # 儲存為圖片
    plt.savefig(f"{filename}.png", bbox_inches='tight', dpi=300)
    plt.close()

In [ ]:
OutputMatrix(Avg_gateError, "Avg_gateError_1Wshots")

In [ ]:
filename = 'matrix_TwirlingError.csv'
np.savetxt(filename, Avg_gateError, delimiter=",", fmt="%.10e")

In [ ]:
Bell_circuit = QuantumCircuit(qreg, creg)
Bell_circuit.h(0)
Bell_circuit.cx(0, 1)
Bell_circuit.draw('mpl')

In [ ]:
def add_twirling_circuit(circuit):
    twir_cir_set = {}
    for first_pauli, second_pauli in pauli_transfer.items():
        twir_circuit = QuantumCircuit(circuit.num_qubits, circuit.num_clbits)
        for instr in circuit.data:
            if instr.operation.name == 'cx':
                twir_circuit = QuantumCircuit.compose(twir_circuit, PauliTwirling[first_pauli])     
                twir_circuit.append(instr.operation, instr.qubits, instr.clbits)
                twir_circuit = QuantumCircuit.compose(twir_circuit, PauliTwirling[second_pauli])
            else:
                twir_circuit.append(instr.operation, instr.qubits, instr.clbits)
        twir_cir_set.update({first_pauli:twir_circuit})    
    return twir_cir_set


In [ ]:
Bell_cir_twir = add_twirling_circuit(Bell_circuit)
# Bell_cir_twir["XY"].draw("mpl")

In [ ]:
def run_bell(times):
    Bell_twir_job = []   
    batched_RdO = []
    batched_Noisy = []
    for i in range(times):      
        # PEC TQG
        batched_TQG = []
        for CirName, Cir in Bell_cir_twir.items():
            CirTran = q.compiler.transpile(Cir, backend=backend, optimization_level=0)
            batched_TQG.append((CirTran, Measure))
        Bell_twir_job.append(estimator.run(batched_TQG))
        # PEC RdO
        RCirTran = q.compiler.transpile(Bell_circuit, backend=backend, optimization_level=0)
        batched_RdO.append((RCirTran, IdealMeasurement['measZZ']))
        # Noisy
        BCirTran = q.compiler.transpile(Bell_circuit, backend=backend, optimization_level=0)
        batched_Noisy.append((BCirTran, Measurement['measZZ']))
    Noisy_job = estimator.run(batched_Noisy)
    RdO_job = estimator.run(batched_RdO)
    return Noisy_job, RdO_job, Bell_twir_job

def avg_expectation(jobset, weight):
    average = []
    for job in jobset:
        result_mitigate = job.result()
        result = [res.data.evs for res in result_mitigate]
        average.append(sum(expct*invc for expct, invc in zip(result, weight)))
    return average

In [ ]:
times = 50
job_noisy, job_rdo, job_tqg = run_bell(times)

In [ ]:
result_noisy = job_noisy.result()
result_rdo = job_rdo.result()
NoisyExpectation = [result_noisy[i].data.evs for i in range(times)]
ReadoutExpectation = [result_rdo[i].data.evs for i in range(times)]
TwoQubitGateExpectation = np.real(avg_expectation(job_tqg, inv_alpha2))

print('Noisy_Expectation:\n', NoisyExpectation)
print('Readout_Expectation:\n', ReadoutExpectation)
print('TwoQubitGate_Expectation:\n', TwoQubitGateExpectation)

In [ ]:
def plot_expectation_histogram_multi(data_list,  # List of tuples: [(name, data_array, color), ...] 
                                     Ideal=0,
                                     bin_width=0.005,
                                     title="Histogram of Expectation Values",
                                     xlabel="Expectation Value",
                                     ylabel="Frequency",
                                     save_path=None):

    # 將所有資料轉為 numpy array
    processed = []
    for name, data, color in data_list:
        data_arr = np.array([x.item() for x in data])
        processed.append((name, data_arr, color))

    # 計算 bins 範圍（依所有資料最大絕對值對稱設 bin）
    max_val = max(abs(arr).max() for (_, arr, _) in processed)
    bins = np.arange(-np.ceil(max_val * 100) / 100 - bin_width,
                     np.ceil(max_val * 100) / 100 + bin_width,
                     bin_width)
    bin_centers = (bins[:-1] + bins[1:]) / 2

    # 建立圖片
    fig, ax = plt.subplots(figsize=(12, 6))
    # 避免重疊
    width = bin_width * (0.9 / len(processed))  

    legend_patches = []
    legend_lines = []
    all_values = []

    for idx, (name, arr, color) in enumerate(processed):
        hist, _ = np.histogram(arr, bins=bins)
        shift = (idx - len(processed) / 2 + 0.5) * width  # 偏移位置
        ax.bar(bin_centers + shift, hist, width=width, color=color)

        # 統計量
        mean_val = np.mean(arr)
        median_val = np.median(arr)
        var_val = np.var(arr)
        std_val = np.std(arr)

        # Mean 虛線
        ax.axvline(mean_val, color=color, linestyle='--')

        # Legend handle
        legend_patches.append(mpatches.Patch(color=color, label=name))
        legend_lines.append(mlines.Line2D([], [], color=color, linestyle='--', 
                                          label=f"{name} Mean: {mean_val:.4f}"))

        # # 文字框
        # max_y = max(hist.max() for (_, a, _) in processed)
        # text_x = (all_values_flat.min() + all_values_flat.max()) / 2
        # text_y = max_y * (0.85 - 0.15 * idx)  # 垂直從上往下排
        # ax.text(text_x, text_y,
        #         f"Median: {median_val:.4f}\nVariance: {var_val:.4f}\nStd Dev: {std_val:.4f}",
        #         color=color, fontsize=10, fontweight='bold',
        #         verticalalignment='top', horizontalalignment='center',
        #         bbox=dict(boxstyle="round,pad=0.4", edgecolor=color, facecolor='white', linewidth=1))

        all_values.append(arr)
        processed[idx] = (name, arr, color, hist, shift,
                          np.median(arr), np.var(arr), np.std(arr))  # 延伸原本的 tuple

    all_values_flat = np.concatenate(all_values)
    text_x = (all_values_flat.min() + all_values_flat.max()) / 2
    max_y = max(hist.max() for (_, _, _, hist, _, _, _, _) in processed)
    # 繪製文字框
    for idx, (name, arr, color, hist, shift, median_val, var_val, std_val) in enumerate(processed):
        text_y = max_y * (0.8 - 0.2 * idx)
        ax.text(text_x, text_y,
                f"Median: {median_val:.4f}\nVariance: {var_val:.4f}\nStd Dev: {std_val:.4f}",
                color=color, fontsize=10, fontweight='bold',
                verticalalignment='top', horizontalalignment='center',
                bbox=dict(boxstyle="round,pad=0.4", edgecolor=color, facecolor='white', linewidth=1))
        
    # X 軸設定
    xticks = np.arange(np.floor(bin_centers.min()*200)/200,
                       np.ceil(bin_centers.max()*200)/200 + bin_width,
                       bin_width)
    ax.set_xticks(xticks)
    ax.set_xticklabels([f'{x:.3f}' for x in xticks], rotation=45)
    ax.set_xlim(all_values_flat.min() - bin_width, all_values_flat.max() + bin_width)

    # 標題與說明
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.grid(True)

    # 圖例
    ax.legend(handles=legend_patches + legend_lines,
              loc='upper right',
              ncol=2, fontsize=10, frameon=True)

    # 理論值
    ax.axvline(Ideal, color='red', linestyle='-')

    # 顯示或儲存
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=500, bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
path = f"diagram/0728_sim_{device_name}_{times}.png"
plot_expectation_histogram_multi([("Noisy", NoisyExpectation, "cornflowerblue"),
                                  ("Readout", ReadoutExpectation, "mediumseagreen"),
                                  ("TwoQubitGate", TwoQubitGateExpectation, "orange")],
                                  Ideal = 1, save_path = path)
